In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/modules/tools/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDb = FAISS.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectorDb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021B7DFD5150>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langchain_tools_search",
                      "Search for informations about tools in langsmith. For any question about tools u must use this.")

In [22]:
retriever_tool.name
retriever_tool.invoke("what are tools")

"They combine a few things:The name of the toolA description of what the tool isJSON schema of what the inputs to the tool areThe function to call Whether the result of a tool should be returned directly to the userIt is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.The simpler the input to a tool is, the easier it is for an LLM to be able to use it.\nMany agents will only work with tools that have a single string input.\n\nSkip to main contentA newer LangChain version is out! Check out the latest version.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat 

In [15]:
## Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name
arxiv.description

'A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.'

In [17]:
tools = [wiki, arxiv,retriever_tool]

## Agents

In [27]:
from dotenv import load_dotenv
import os 
from langchain_google_genai import GoogleGenerativeAI
from langchain import hub

In [25]:
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2") # this solved the problem of the project not appearing in langsmith dashboard
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [37]:
from langchain_community.chat_models import ChatOllama
## Load Ollama LAMA2 LLM model
llm=ChatOllama(model="llama3.1")
llm

ChatOllama(model='llama3.1')

In [26]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash")

In [28]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [40]:
## Agent executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [41]:
agent_executor.invoke({"input":"Tell me about tools in langchain"})



> Entering new AgentExecutor chain...
LangChain is an open-source framework for building conversational AI models, and it comes with a wide range of pre-built tools that can be used to enhance the functionality of your chatbots. Here are some of the key tools available in LangChain:

1. **LLaMA**: This is a large language model (LLM) inference tool that allows you to use pre-trained LLMs within your LangChain models. LLaMA provides a flexible interface for integrating various LLMs, making it easier to experiment with different models.
2. **LLaMA-Chat**: A simple chatbot tool built on top of the LLaMA model. This tool enables you to build conversational interfaces using pre-trained language models and can be used as a starting point for more complex applications.
3. **Chain**: This is the core tool in LangChain, responsible for building and managing conversation flows. Chain provides a graph-based API that allows you to create custom chatbot experiences by linking together various com

{'input': 'Tell me about tools in langchain',
 'output': "LangChain is an open-source framework for building conversational AI models, and it comes with a wide range of pre-built tools that can be used to enhance the functionality of your chatbots. Here are some of the key tools available in LangChain:\n\n1. **LLaMA**: This is a large language model (LLM) inference tool that allows you to use pre-trained LLMs within your LangChain models. LLaMA provides a flexible interface for integrating various LLMs, making it easier to experiment with different models.\n2. **LLaMA-Chat**: A simple chatbot tool built on top of the LLaMA model. This tool enables you to build conversational interfaces using pre-trained language models and can be used as a starting point for more complex applications.\n3. **Chain**: This is the core tool in LangChain, responsible for building and managing conversation flows. Chain provides a graph-based API that allows you to create custom chatbot experiences by linkin

In [44]:
agent_executor.invoke({"input":"Tell me about ReiID in computer vision"})



> Entering new AgentExecutor chain...
ReID (Re-Identification) is a subfield of computer vision that focuses on identifying the same individual across non-overlapping camera views or images. The goal of ReID is to associate objects, specifically people, between different cameras or scenes, even if they are not directly connected.

In the context of surveillance and security applications, ReID aims to help identify individuals who may be recognized from a specific camera view in another part of the system or across multiple cameras. This can be particularly useful for tracking suspects, identifying known individuals, or monitoring crowd behavior.

The tasks involved in ReID include:

1. **Person detection**: Locating individuals within images or videos.
2. **Person re-identification**: Associating individuals between non-overlapping camera views or images.
3. **Person tracking**: Following an individual across multiple cameras or scenes.

ReID is a challenging problem due to variation

{'input': 'Tell me about ReiID in computer vision',
 'output': 'ReID (Re-Identification) is a subfield of computer vision that focuses on identifying the same individual across non-overlapping camera views or images. The goal of ReID is to associate objects, specifically people, between different cameras or scenes, even if they are not directly connected.\n\nIn the context of surveillance and security applications, ReID aims to help identify individuals who may be recognized from a specific camera view in another part of the system or across multiple cameras. This can be particularly useful for tracking suspects, identifying known individuals, or monitoring crowd behavior.\n\nThe tasks involved in ReID include:\n\n1. **Person detection**: Locating individuals within images or videos.\n2. **Person re-identification**: Associating individuals between non-overlapping camera views or images.\n3. **Person tracking**: Following an individual across multiple cameras or scenes.\n\nReID is a ch